# Liquid Handler Program for GatorBio Assay
Generated from: GatorBio Assay Form.xlsm
Generated on: 2025-11-11 09:14:28
This notebook prepares assay and Max Plate (ProbeInfo) dilutions using pylabrobot.

## Setup Machine

In [ ]:
%load_ext autoreload
%autoreload 2
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import STARBackend
from pylabrobot.resources import Deck, Coordinate
from pylabrobot.liquid_handling import Strictness, set_strictness
from pylabrobot.resources.hamilton import STARDeck
import time

lh = LiquidHandler(backend=STARBackend(read_timeout=600), deck=STARDeck(core_grippers="1000uL-5mL-on-waste"))
await lh.setup(skip_iswap=True)
set_strictness(Strictness.STRICT)

## Set Variables
Change these values for your experiment.

In [ ]:
# Assay plate parameters
FINAL_VOLUME = 200  # µL transferred to assay 96-well plate
DILUTION_VOLUME = 300  # µL per dilution in deep well plate

# Max Plate parameters
MAX_PLATE_FINAL_VOLUME = 200  # µL transferred to Max Plate
MAX_PLATE_DILUTION_VOLUME = 300  # µL per dilution in Max Plate deep well

# Mixing and flow parameters
MIX_CYCLES = 3
FLOW_RATE = 100  # µL/sec
CHANGE_TIPS_BETWEEN_DILUTIONS = True

# Assay initial stock concentrations (µg/mL)
INITIAL_CONC_S_3_12 = 1.0  # µg/mL - Stock concentration for 3-12INITIAL_CONC_S_4_2 = 1.0  # µg/mL - Stock concentration for 4-2INITIAL_CONC_S_4_3 = 1.0  # µg/mL - Stock concentration for 4-3INITIAL_CONC_S_4_4 = 1.0  # µg/mL - Stock concentration for 4-4INITIAL_CONC_S_6_1 = 1.0  # µg/mL - Stock concentration for 6-1INITIAL_CONC_S_6_2 = 1.0  # µg/mL - Stock concentration for 6-2INITIAL_CONC_EPO_R = 12.85  # µg/mL - Stock concentration for EPO-RINITIAL_CONC_WT = 1.0  # µg/mL - Stock concentration for WT
# Max Plate initial stock concentrations (µg/mL)
INITIAL_CONC_MAX_P_6_3 = 1.0  # µg/mL - Stock concentration for Max Plate 6-3INITIAL_CONC_MAX_P_6_4 = 1.0  # µg/mL - Stock concentration for Max Plate 6-4INITIAL_CONC_MAX_P_6_5 = 1.0  # µg/mL - Stock concentration for Max Plate 6-5INITIAL_CONC_MAX_P_6_6 = 1.0  # µg/mL - Stock concentration for Max Plate 6-6

## Stock Volume Requirements
Estimated minimum volumes to load in each 50 mL tube (add extra for dead volume):
- `stock_buffer`: 33.30 mL (≈ 33301 µL)
- `stock_neutralization`: 1.60 mL (≈ 1600 µL)
- `stock_regeneration`: 1.60 mL (≈ 1600 µL)

Probe wells buffer usage is included in the `stock_buffer` total.


## Sample Stock Placement
- `3-12` → `dilution_plate['A1']` (0.40 mL) (includes reserve for next dilution)
- `4-2` → `dilution_plate['B1']` (0.40 mL) (includes reserve for next dilution)
- `4-3` → `dilution_plate['C1']` (0.40 mL) (includes reserve for next dilution)
- `4-4` → `dilution_plate['D1']` (0.40 mL) (includes reserve for next dilution)
- `6-1` → `dilution_plate['E1']` (0.40 mL) (includes reserve for next dilution)
- `6-2` → `dilution_plate['F1']` (0.40 mL) (includes reserve for next dilution)
- `EPO-R` → `dilution_plate['C5']` (2.40 mL)
- `WT` → `dilution_plate['D5']` (0.40 mL) (includes reserve for next dilution)
- `6-3` → `dilution_plate['G1']` (0.40 mL) (includes reserve for next dilution)
- `6-4` → `dilution_plate['H1']` (0.40 mL) (includes reserve for next dilution)
- `6-5` → `dilution_plate['A5']` (0.40 mL) (includes reserve for next dilution)
- `6-6` → `dilution_plate['B5']` (0.40 mL) (includes reserve for next dilution)


In [ ]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cor_96_wellplate_360ul_Fb,
    nest_96_wellplate_2mL_Vb,
    hamilton_96_tiprack_1000uL,
    Tube_CAR_32_A00,
    hamilton_tube_carrier_12_b00,
    Cor_Falcon_tube_50mL_Vb,
)
from pylabrobot.liquid_handling.standard import Mix

lh.deck.get_resource("trash_core96").location = Coordinate(-260, 106, 216.4)

# Tips
tip_car = TIP_CAR_480_A00(name="tip_carrier")
tip_car[0] = hamilton_96_tiprack_1000uL(name="main_tips")
lh.deck.assign_child_resource(tip_car, rails=7)

TIP_POSITIONS = [
    'A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1',
    'A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2',
    'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3',
    'A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4',
    'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5',
    'A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6',
    'A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7',
    'A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8',
    'A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9',
    'A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10',
    'A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11',
    'A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12',
]
tip_position_index = 0

# Plates
dilution_plate_names = ['dilution_plate']
plt_car = PLT_CAR_L5AC_A00(name="plate_carrier")
plt_car[0] = Cor_96_wellplate_360ul_Fb(name="final_plate")  # Assay plate
plt_car[1] = nest_96_wellplate_2mL_Vb(name=dilution_plate_names[0])  # Shared dilutions
plt_car[2] = Cor_96_wellplate_360ul_Fb(name="max_plate_final")  # Max Plate
if len(dilution_plate_names) > 1:
    plt_car[3] = nest_96_wellplate_2mL_Vb(name=dilution_plate_names[1])  # Additional dilutions
lh.deck.assign_child_resource(plt_car, rails=1)

# 50 mL stock tubes
stock_resources = ['stock_buffer', 'stock_neutralization', 'stock_regeneration']
tube_car = hamilton_tube_carrier_12_b00(name="tube_carrier")
for i, resource_name in enumerate(stock_resources):
    if i >= 12:
        print(f"Warning: Not enough tube positions for {resource_name}")
        continue
    tube_car[11 - i] = Cor_Falcon_tube_50mL_Vb(name=resource_name)
lh.deck.assign_child_resource(tube_car, rails=35)

lh.summary()

## Helper Functions

In [ ]:
def next_tip_positions(count):
    global tip_position_index
    if tip_position_index + count > len(TIP_POSITIONS):
        raise RuntimeError('Not enough tips available for this run.')
    positions = TIP_POSITIONS[tip_position_index:tip_position_index + count]
    tip_position_index += count
    return positions

def get_tip_resources(positions):
    return [lh.deck.get_resource('main_tips')[pos] for pos in positions]

def group_wells_by_column(wells):
    columns = {}
    for well in wells:
        column = ''.join([ch for ch in well if ch.isdigit()])
        columns.setdefault(column, []).append(well)
    for column in columns:
        columns[column].sort()
    return columns

async def serial_dilution(source_plate_name, target_plate_name, source_well, target_well, transfer_volume, final_volume, mix_cycles=MIX_CYCLES):
    """Transfer an aliquot from source to target and mix for serial dilution."""
    source_plate = lh.deck.get_resource(source_plate_name)
    target_plate = lh.deck.get_resource(target_plate_name)
    tip_positions = next_tip_positions(1)
    tips = get_tip_resources(tip_positions)
    await lh.pick_up_tips(tips)
    await lh.aspirate([source_plate[source_well]], vols=[transfer_volume], flow_rates=[FLOW_RATE])
    await lh.dispense([target_plate[target_well]], vols=[transfer_volume], flow_rates=[FLOW_RATE])
    for _ in range(mix_cycles):
        mix_volume = min(final_volume * 0.8, final_volume)
        await lh.aspirate([target_plate[target_well]], vols=[mix_volume], flow_rates=[FLOW_RATE])
        await lh.dispense([target_plate[target_well]], vols=[mix_volume], flow_rates=[FLOW_RATE], offsets=[Coordinate(0, 0, 5)])
    await lh.drop_tips([lh.deck.get_resource('trash')]*len(tips))

async def transfer_from_tube_to_plate(tube_resource_name, plate_name, target_well, volume):
    """Transfer liquid from a 50 mL tube to a plate well"""
    tube_resource = lh.deck.get_resource(tube_resource_name)
    plate = lh.deck.get_resource(plate_name)
    tip_positions = next_tip_positions(1)
    tips = get_tip_resources(tip_positions)
    await lh.pick_up_tips(tips)
    await lh.aspirate([tube_resource], vols=[volume], flow_rates=[FLOW_RATE])
    await lh.dispense([plate[target_well]], vols=[volume], flow_rates=[FLOW_RATE])
    await lh.drop_tips([lh.deck.get_resource('trash')]*len(tips))

async def load_plate_columns_from_stock(stock_resource_name, plate_name, wells, volume_per_well):
    if not wells:
        return
    wells_by_column = group_wells_by_column(wells)
    for column in sorted(wells_by_column.keys(), key=lambda c: int(c)):
        column_wells = wells_by_column[column]
        if len(column_wells) == 8:
            volumes = [volume_per_well] * 8
            await multi_channel_transfer_from_tube(stock_resource_name, plate_name, int(column), volumes)
        else:
            for well in column_wells:
                await transfer_from_tube_to_plate(stock_resource_name, plate_name, well, volume_per_well)

def build_channel_indices(volumes):
    return [idx for idx, vol in enumerate(volumes) if vol and vol > 0]

async def multi_channel_transfer_from_tube(tube_resource_name, plate_name, column_number, volumes, change_tips=True):
    channel_indices = build_channel_indices(volumes)
    if not channel_indices:
        return
    tube = lh.deck.get_resource(tube_resource_name)
    plate = lh.deck.get_resource(plate_name)
    tip_positions = next_tip_positions(len(channel_indices))
    tip_resources = get_tip_resources(tip_positions)
    await lh.pick_up_tips(tip_resources)
    source_containers = [tube for _ in channel_indices]
    volumes_list = [volumes[idx] for idx in channel_indices]
    await lh.aspirate(source_containers, vols=volumes_list, use_channels=channel_indices, flow_rates=[FLOW_RATE] * len(channel_indices))
    target_positions = [f"{chr(ord('A') + idx)}{column_number}" for idx in channel_indices]
    targets = [plate[pos] for pos in target_positions]
    await lh.dispense(targets, vols=volumes_list, use_channels=channel_indices, flow_rates=[FLOW_RATE] * len(channel_indices))
    await lh.drop_tips([lh.deck.get_resource('trash')]*len(channel_indices))

async def multi_channel_serial_dilution(source_plate_name, target_plate_name, source_column, target_column, transfer_volumes, total_volumes, sample_labels, change_tips, reuse_state):
    channel_indices = build_channel_indices(transfer_volumes)
    if not channel_indices:
        return
    source_plate = lh.deck.get_resource(source_plate_name)
    target_plate = lh.deck.get_resource(target_plate_name)
    source_positions = [f"{chr(ord('A') + idx)}{source_column}" for idx in channel_indices]
    target_positions = [f"{chr(ord('A') + idx)}{target_column}" for idx in channel_indices]
    if change_tips or not reuse_state.get('loaded'):
        tip_positions = next_tip_positions(len(channel_indices))
        tip_resources = get_tip_resources(tip_positions)
        await lh.pick_up_tips(tip_resources)
        reuse_state['loaded'] = True
        reuse_state['tip_resources'] = tip_resources
    else:
        tip_resources = reuse_state['tip_resources']
    source_containers = [source_plate[pos] for pos in source_positions]
    transfer_list = [transfer_volumes[idx] for idx in channel_indices]
    await lh.aspirate(source_containers, vols=transfer_list, use_channels=channel_indices, flow_rates=[FLOW_RATE] * len(channel_indices))
    target_containers = [target_plate[pos] for pos in target_positions]
    mix_args = None
    if MIX_CYCLES and MIX_CYCLES > 0:
        mix_args = []
        for idx, channel_idx in enumerate(channel_indices):
            channel_total = total_volumes[channel_idx] or transfer_list[idx]
            mix_volume = channel_total * 0.8 if channel_total else transfer_list[idx]
            mix_args.append(Mix(volume=mix_volume, repetitions=MIX_CYCLES, flow_rate=FLOW_RATE))
    await lh.dispense(target_containers, vols=transfer_list, use_channels=channel_indices, flow_rates=[FLOW_RATE] * len(channel_indices), mix=mix_args)
    for idx, channel_idx in enumerate(channel_indices):
        label = sample_labels[channel_idx]
        if label:
            print(f"{label}: column {source_column}->{target_column} dilution complete")
    if change_tips:
        await lh.drop_tips([lh.deck.get_resource('trash')]*len(channel_indices))
        reuse_state['loaded'] = False
        reuse_state['tip_resources'] = []
    else:
        reuse_state['tip_resources'] = tip_resources

async def transfer_to_final_plate(source_plate_name, target_plate_name, source_well, target_well, volume):
    """Transfer liquid from source plate to target plate"""
    source_plate = lh.deck.get_resource(source_plate_name)
    target_plate = lh.deck.get_resource(target_plate_name)
    tip_positions = next_tip_positions(1)
    tips = get_tip_resources(tip_positions)
    await lh.pick_up_tips(tips)
    await lh.aspirate([source_plate[source_well]], vols=[volume], flow_rates=[FLOW_RATE])
    await lh.dispense([target_plate[target_well]], vols=[volume], flow_rates=[FLOW_RATE])
    await lh.drop_tips([lh.deck.get_resource('trash')]*len(tips))


## Execute Liquid Handling

In [ ]:
print("Starting liquid handling...")
print("Preparing assay plate dilutions...")

print("Preparing Max Plate dilutions...")

print("Starting sample stock transfers...")
# Ensure 3-12 sample stock (399.9 µL) is pre-loaded into dilution_plate well A1
# Ensure 4-2 sample stock (399.9 µL) is pre-loaded into dilution_plate well B1
# Ensure 4-3 sample stock (399.9 µL) is pre-loaded into dilution_plate well C1
# Ensure 4-4 sample stock (399.9 µL) is pre-loaded into dilution_plate well D1
# Ensure 6-1 sample stock (399.9 µL) is pre-loaded into dilution_plate well E1
# Ensure 6-2 sample stock (399.9 µL) is pre-loaded into dilution_plate well F1
# Ensure EPO-R load stock (2400.0 µL) is pre-loaded into dilution_plate well C5
# Ensure WT sample stock (399.9 µL) is pre-loaded into dilution_plate well D5
# Ensure 6-3 sample stock (399.9 µL) is pre-loaded into dilution_plate well G1
# Ensure 6-4 sample stock (399.9 µL) is pre-loaded into dilution_plate well H1
# Ensure 6-5 sample stock (399.9 µL) is pre-loaded into dilution_plate well A5
# Ensure 6-6 sample stock (399.9 µL) is pre-loaded into dilution_plate well B5

print("Prefilling buffer into dilution plate columns...")
await multi_channel_transfer_from_tube('stock_buffer', 'dilution_plate', 2, [300.1, 300.1, 300.1, 300.1, 300.1, 300.1, 300.1, 300.1])
await multi_channel_transfer_from_tube('stock_buffer', 'dilution_plate', 3, [300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0])
await multi_channel_transfer_from_tube('stock_buffer', 'dilution_plate', 4, [300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0])
await multi_channel_transfer_from_tube('stock_buffer', 'dilution_plate', 6, [300.1, 300.1, 0.0, 300.1, 0.0, 0.0, 0.0, 0.0])
await multi_channel_transfer_from_tube('stock_buffer', 'dilution_plate', 7, [300.0, 300.0, 0.0, 300.0, 0.0, 0.0, 0.0, 0.0])
await multi_channel_transfer_from_tube('stock_buffer', 'dilution_plate', 8, [300.0, 300.0, 0.0, 300.0, 0.0, 0.0, 0.0, 0.0])

print("Starting serial dilutions...")
tip_reuse_state = {"loaded": False}
await multi_channel_serial_dilution('dilution_plate', 'dilution_plate', 1, 2, [99.9, 99.9, 99.9, 99.9, 99.9, 99.9, 99.9, 99.9], [400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0], ["3-12", "4-2", "4-3", "4-4", "6-1", "6-2", "6-3", "6-4"], CHANGE_TIPS_BETWEEN_DILUTIONS, tip_reuse_state)
await multi_channel_serial_dilution('dilution_plate', 'dilution_plate', 2, 3, [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0], [400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0], ["3-12", "4-2", "4-3", "4-4", "6-1", "6-2", "6-3", "6-4"], CHANGE_TIPS_BETWEEN_DILUTIONS, tip_reuse_state)
await multi_channel_serial_dilution('dilution_plate', 'dilution_plate', 3, 4, [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0], [400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0], ["3-12", "4-2", "4-3", "4-4", "6-1", "6-2", "6-3", "6-4"], CHANGE_TIPS_BETWEEN_DILUTIONS, tip_reuse_state)
await multi_channel_serial_dilution('dilution_plate', 'dilution_plate', 5, 6, [99.9, 99.9, 0.0, 99.9, 0.0, 0.0, 0.0, 0.0], [400.0, 400.0, 0.0, 400.0, 0.0, 0.0, 0.0, 0.0], ["6-5", "6-6", "", "WT", "", "", "", ""], CHANGE_TIPS_BETWEEN_DILUTIONS, tip_reuse_state)
await multi_channel_serial_dilution('dilution_plate', 'dilution_plate', 6, 7, [100.0, 100.0, 0.0, 100.0, 0.0, 0.0, 0.0, 0.0], [400.0, 400.0, 0.0, 400.0, 0.0, 0.0, 0.0, 0.0], ["6-5", "6-6", "", "WT", "", "", "", ""], CHANGE_TIPS_BETWEEN_DILUTIONS, tip_reuse_state)
await multi_channel_serial_dilution('dilution_plate', 'dilution_plate', 7, 8, [100.0, 100.0, 0.0, 100.0, 0.0, 0.0, 0.0, 0.0], [400.0, 400.0, 0.0, 400.0, 0.0, 0.0, 0.0, 0.0], ["6-5", "6-6", "", "WT", "", "", "", ""], CHANGE_TIPS_BETWEEN_DILUTIONS, tip_reuse_state)
if not CHANGE_TIPS_BETWEEN_DILUTIONS and tip_reuse_state['loaded']:
    await lh.drop_tips([lh.deck.get_resource('trash')] * len(tip_reuse_state['tip_resources']))
    tip_reuse_state['loaded'] = False
    tip_reuse_state['tip_resources'] = []


print("Processing Assay column 1")
print("Processing Assay column 5")
# EPO-R reuses C5; no additional buffer or stock required
# EPO-R reuses C5; no additional buffer or stock required
# EPO-R reuses C5; no additional buffer or stock required
# EPO-R reuses C5; no additional buffer or stock required
# EPO-R reuses C5; no additional buffer or stock required
# EPO-R reuses C5; no additional buffer or stock required
# EPO-R reuses C5; no additional buffer or stock required
print("Processing Max column 1")
print("Processing Max column 5")
print("Loading shared reagents into assay final plate...")
await load_plate_columns_from_stock('stock_buffer', 'final_plate', ["A1", "B1", "C1", "D1", "E1", "F1", "G1", "H1", "A3", "B3", "C3", "D3", "E3", "F3", "G3", "H3", "A5", "B5", "C5", "D5", "E5", "F5", "G5", "H5", "A9", "B9", "C9", "D9", "E9", "F9", "G9", "H9", "A11", "B11", "C11", "D11", "E11", "F11", "G11", "H11"], FINAL_VOLUME)
print("stock_buffer: loaded 40 assay wells")
await load_plate_columns_from_stock('stock_neutralization', 'final_plate', ["A7", "B7", "C7", "D7", "E7", "F7", "G7", "H7"], FINAL_VOLUME)
print("stock_neutralization: loaded 8 assay wells")
await load_plate_columns_from_stock('stock_regeneration', 'final_plate', ["A6", "B6", "C6", "D6", "E6", "F6", "G6", "H6"], FINAL_VOLUME)
print("stock_regeneration: loaded 8 assay wells")

print("Loading shared reagents into Max Plate...")
await load_plate_columns_from_stock('stock_buffer', 'max_plate_final', ["A1", "B1", "C1", "D1", "E1", "F1", "G1", "H1", "A2", "B2", "C2", "D2", "E2", "F2", "G2", "H2", "A4", "B4", "C4", "D4", "E4", "F4", "G4", "H4", "A6", "B6", "C6", "D6", "E6", "F6", "G6", "H6", "C7", "D7", "E7", "F7", "G7", "A8", "B8", "C8", "D8", "E8", "F8", "G8", "H8", "A9", "B9", "C9", "D9", "E9", "F9", "G9", "H9", "A10", "B10", "C10", "D10", "E10", "F10", "G10", "H10", "A11", "B11", "C11", "D11", "E11", "F11", "G11", "H11", "A12", "B12", "C12", "D12", "E12", "F12", "G12", "H12"], MAX_PLATE_FINAL_VOLUME)
print("stock_buffer: loaded 77 Max Plate wells")

print("Transferring assay dilutions to final plate...")
await transfer_to_final_plate('dilution_plate', 'final_plate', 'A1', 'E4', FINAL_VOLUME)  # 3-12
await transfer_to_final_plate('dilution_plate', 'final_plate', 'A2', 'F4', FINAL_VOLUME)  # 3-12
await transfer_to_final_plate('dilution_plate', 'final_plate', 'A3', 'G4', FINAL_VOLUME)  # 3-12
await transfer_to_final_plate('dilution_plate', 'final_plate', 'A4', 'A8', FINAL_VOLUME)  # 3-12
await transfer_to_final_plate('dilution_plate', 'final_plate', 'B1', 'B8', FINAL_VOLUME)  # 4-2
await transfer_to_final_plate('dilution_plate', 'final_plate', 'B2', 'C8', FINAL_VOLUME)  # 4-2
await transfer_to_final_plate('dilution_plate', 'final_plate', 'B3', 'D8', FINAL_VOLUME)  # 4-2
await transfer_to_final_plate('dilution_plate', 'final_plate', 'B4', 'E8', FINAL_VOLUME)  # 4-2
await transfer_to_final_plate('dilution_plate', 'final_plate', 'C1', 'F8', FINAL_VOLUME)  # 4-3
await transfer_to_final_plate('dilution_plate', 'final_plate', 'C2', 'G8', FINAL_VOLUME)  # 4-3
await transfer_to_final_plate('dilution_plate', 'final_plate', 'C3', 'A10', FINAL_VOLUME)  # 4-3
await transfer_to_final_plate('dilution_plate', 'final_plate', 'C4', 'B10', FINAL_VOLUME)  # 4-3
await transfer_to_final_plate('dilution_plate', 'final_plate', 'D1', 'C10', FINAL_VOLUME)  # 4-4
await transfer_to_final_plate('dilution_plate', 'final_plate', 'D2', 'D10', FINAL_VOLUME)  # 4-4
await transfer_to_final_plate('dilution_plate', 'final_plate', 'D3', 'E10', FINAL_VOLUME)  # 4-4
await transfer_to_final_plate('dilution_plate', 'final_plate', 'D4', 'F10', FINAL_VOLUME)  # 4-4
await transfer_to_final_plate('dilution_plate', 'final_plate', 'E1', 'G10', FINAL_VOLUME)  # 6-1
await transfer_to_final_plate('dilution_plate', 'final_plate', 'E2', 'A12', FINAL_VOLUME)  # 6-1
await transfer_to_final_plate('dilution_plate', 'final_plate', 'E3', 'B12', FINAL_VOLUME)  # 6-1
await transfer_to_final_plate('dilution_plate', 'final_plate', 'E4', 'C12', FINAL_VOLUME)  # 6-1
await transfer_to_final_plate('dilution_plate', 'final_plate', 'F1', 'D12', FINAL_VOLUME)  # 6-2
await transfer_to_final_plate('dilution_plate', 'final_plate', 'F2', 'E12', FINAL_VOLUME)  # 6-2
await transfer_to_final_plate('dilution_plate', 'final_plate', 'F3', 'F12', FINAL_VOLUME)  # 6-2
await transfer_to_final_plate('dilution_plate', 'final_plate', 'F4', 'G12', FINAL_VOLUME)  # 6-2
await transfer_to_final_plate('dilution_plate', 'final_plate', 'C5', 'A2', FINAL_VOLUME)  # EPO-R
await transfer_to_final_plate('dilution_plate', 'final_plate', 'C5', 'B2', FINAL_VOLUME)  # EPO-R
await transfer_to_final_plate('dilution_plate', 'final_plate', 'C5', 'C2', FINAL_VOLUME)  # EPO-R
await transfer_to_final_plate('dilution_plate', 'final_plate', 'C5', 'D2', FINAL_VOLUME)  # EPO-R
await transfer_to_final_plate('dilution_plate', 'final_plate', 'C5', 'E2', FINAL_VOLUME)  # EPO-R
await transfer_to_final_plate('dilution_plate', 'final_plate', 'C5', 'F2', FINAL_VOLUME)  # EPO-R
await transfer_to_final_plate('dilution_plate', 'final_plate', 'C5', 'G2', FINAL_VOLUME)  # EPO-R
await transfer_to_final_plate('dilution_plate', 'final_plate', 'C5', 'H2', FINAL_VOLUME)  # EPO-R
await transfer_to_final_plate('dilution_plate', 'final_plate', 'D5', 'A4', FINAL_VOLUME)  # WT
await transfer_to_final_plate('dilution_plate', 'final_plate', 'D6', 'B4', FINAL_VOLUME)  # WT
await transfer_to_final_plate('dilution_plate', 'final_plate', 'D7', 'C4', FINAL_VOLUME)  # WT
await transfer_to_final_plate('dilution_plate', 'final_plate', 'D8', 'D4', FINAL_VOLUME)  # WT

print("Transferring Max Plate dilutions to final plate...")
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'G1', 'A3', MAX_PLATE_FINAL_VOLUME)  # 6-3
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'G2', 'B3', MAX_PLATE_FINAL_VOLUME)  # 6-3
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'G3', 'C3', MAX_PLATE_FINAL_VOLUME)  # 6-3
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'G4', 'D3', MAX_PLATE_FINAL_VOLUME)  # 6-3
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'H1', 'E3', MAX_PLATE_FINAL_VOLUME)  # 6-4
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'H2', 'F3', MAX_PLATE_FINAL_VOLUME)  # 6-4
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'H3', 'G3', MAX_PLATE_FINAL_VOLUME)  # 6-4
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'H4', 'A5', MAX_PLATE_FINAL_VOLUME)  # 6-4
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'A5', 'B5', MAX_PLATE_FINAL_VOLUME)  # 6-5
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'A6', 'C5', MAX_PLATE_FINAL_VOLUME)  # 6-5
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'A7', 'D5', MAX_PLATE_FINAL_VOLUME)  # 6-5
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'A8', 'E5', MAX_PLATE_FINAL_VOLUME)  # 6-5
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'B5', 'F5', MAX_PLATE_FINAL_VOLUME)  # 6-6
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'B6', 'G5', MAX_PLATE_FINAL_VOLUME)  # 6-6
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'B7', 'A7', MAX_PLATE_FINAL_VOLUME)  # 6-6
await transfer_to_final_plate('dilution_plate', 'max_plate_final', 'B8', 'B7', MAX_PLATE_FINAL_VOLUME)  # 6-6

print("Liquid handling complete!")